In [1]:
import load_dataset
import network
import sys
import numpy as np
import keras as ks
from keras.utils import np_utils
import time

Using TensorFlow backend.


In [2]:
lyrics = load_dataset.load_dataset()

Loading dataset...
Done.
Extracting relevant data...
Removing invalid characters...
Done.


In [3]:
#lyrics.info()
genre_counts = lyrics["genre"].value_counts()
# Try to find an equal number of samples per genre,
# otherwise the result is always rock
sub_data_sets={}
for g in lyrics["genre"].unique():
    sub_data_sets[g] = lyrics[lyrics["genre"]==g]
    print("Genre {} has {} lyrics.".format(g, np.sum(lyrics["genre"]==g)))
min_lyrics_per_genre = 5935

tmp=None

for g in sub_data_sets:
    if len(sub_data_sets[g]) < min_lyrics_per_genre:
        print("Skipped genre {} due to insufficient data".format(g))
        continue
    if tmp is None:
        tmp = sub_data_sets[g][:min_lyrics_per_genre]
    else:
        tmp = tmp.append(sub_data_sets[g][:min_lyrics_per_genre])
        
lyrics = tmp.sample(frac=1)
lyrics.info()

Genre Pop has 49444 lyrics.
Genre Hip-Hop has 33965 lyrics.
Genre Rock has 131377 lyrics.
Genre Metal has 28408 lyrics.
Genre Country has 17286 lyrics.
Genre Jazz has 17147 lyrics.
Genre Electronic has 16205 lyrics.
Genre Folk has 3241 lyrics.
Genre R&B has 5935 lyrics.
Genre Indie has 5732 lyrics.
Skipped genre Folk due to insufficient data
Skipped genre Indie due to insufficient data
<class 'pandas.core.frame.DataFrame'>
Int64Index: 47480 entries, 99255 to 33758
Data columns (total 6 columns):
index     47480 non-null int64
song      47480 non-null object
year      47480 non-null int64
artist    47480 non-null object
genre     47480 non-null object
lyrics    47480 non-null object
dtypes: int64(2), object(4)
memory usage: 2.5+ MB


In [4]:
max_seq_length=100
max_seq_count=1000000
(tokenizer, data_input_train, data_labels_train, data_input_test, data_labels_test, label_classes_to_index) = load_dataset.preprocess_dataset(lyrics, max_seq_length, max_seq_count)
index_to_label_class = {v: k for k, v in label_classes_to_index.items()}

Tokenize dataset...
Done.
Loaded 47480 documents.
Convert docs to sequences of length 100...
Done.
Generated 945739 sequences from 27751 documents.
Skipped 19729 docs.


In [5]:
(embeddings_words, embeddings_vec_size) = load_dataset.load_embeddings()
#embeddings_vec_size = 32
#embeddings_matrix = []

Load word embeddings...
Done.


In [6]:
(embeddings_matrix, idx_to_word_map) = load_dataset.glove_to_matrix(embeddings_words, tokenizer)

Converting embeddings to weight matrix...
Skipped 61430 unknown words.
['youve', 'theyll', 'cmon', 'tãº', 'itll', 'theyve', 'therell', 'motherfuckin', 'mã¡s', 'shouldnt', 'shouldve', 'thatll', 'corazã³n', 'motherfucking', 'gyal', 'ã\xa0', 'ayy', 'muthafucka', 'werent', 'sã©', 'theyd', 'asã\xad', 'muthafuckin', 'grindin', 'knowin', 'finna', 'dibby', 'wouldve', 'wholl', 'fã¼r', 'thuggin', 'couldve', 'î½î±', 'ã©', 'quã©', 'sã³lo', 'oclock', 'nã£o', 'wizzy', 'prayin', 'mã\xad', 'aquã\xad', 'lã\xa0', 'whyd', 'showin', 'estã¡', 'hadnt', 'dã\xada', 'lettin', 'einfach', 'listenin', 'stuntin', 'mothafuckin', 'serã¡', 'wonderin', 'bã¼ck', 'gehen', 'ruggish', 'corazn', 'whatchu', 'â¦', 'countin', 'muthaphukkin', 'whod', 'vocãª', 'whoah', 'mustve', 'haffi', 'killas', 'aight', 'normani', 'whyall', 'memries', 'turnt', 'anotha', 'î¼î¿ï\x85', 'tambiã©n', 'slangin', 'hearin', 'piã¹', 'dã\xadas', 'quemo', 'floatin', 'ë\x82\x9c', 'estã¡s', 'î¼îµ', 'mã\xada', 'breathin', 'catchin', 'tryina', 'stackin', 'a

In [7]:
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size:" , vocab_size)
print("Inputs Shape:" , data_input_train.shape)
print("Labels Shape:" , data_labels_train.shape)
print("Inputs Shape:" , data_input_test.shape)
print("Labels Shape:" , data_labels_test.shape)
print("Classes are:")
print(label_classes_to_index.keys())

Vocabulary Size: 116733
Inputs Shape: (851165, 100)
Labels Shape: (851165, 8)
Inputs Shape: (94574, 100)
Labels Shape: (94574, 8)
Classes are:
dict_keys(['Country', 'Pop', 'Rock', 'R&B', 'Hip-Hop', 'Electronic', 'Jazz', 'Metal'])


In [8]:
model = network.get_network(max_seq_length, data_labels_train.shape[1], embeddings_matrix, tokenizer, embeddings_vec_size)
model.summary()

Building network...
Done.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          11673300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 40)                22560     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 328       
Total params: 11,696,188
Trainable params: 22,888
Non-trainable params: 11,673,300
_________________________________________________________________


In [9]:
name = "new_20_default_embedd_32_subset_1000000_words_100"

callbacks = [
    ks.callbacks.TensorBoard("./logs/{}".format(name), write_graph=True, write_grads=False),
    ks.callbacks.ModelCheckpoint("./models/{}".format(name + ".dat"), save_best_only=True)
]

# serialize model to JSON
model_json = model.to_json()
with open("./models/{}".format(name + ".json"), "w") as json_file:
    json_file.write(model_json)

In [10]:
model.fit(data_input_train, data_labels_train, epochs=15, batch_size=2048, validation_data=(data_input_test, data_labels_test), callbacks=callbacks)

Train on 851165 samples, validate on 94574 samples
Epoch 1/15
851165/851165 [==============================] - 298s 350us/step - loss: 1.7483 - categorical_accuracy: 0.3909 - val_loss: 1.6569 - val_categorical_accuracy: 0.4146
Epoch 2/15
851165/851165 [==============================] - 297s 349us/step - loss: 1.6423 - categorical_accuracy: 0.4356 - val_loss: 1.6438 - val_categorical_accuracy: 0.4193
Epoch 3/15
851165/851165 [==============================] - 295s 346us/step - loss: 1.6088 - categorical_accuracy: 0.4451 - val_loss: 1.5905 - val_categorical_accuracy: 0.4339
Epoch 4/15
851165/851165 [==============================] - 294s 346us/step - loss: 1.5450 - categorical_accuracy: 0.4649 - val_loss: 1.5262 - val_categorical_accuracy: 0.4584
Epoch 5/15
851165/851165 [==============================] - 290s 341us/step - loss: 1.4973 - categorical_accuracy: 0.4792 - val_loss: 1.4974 - val_categorical_accuracy: 0.4668
Epoch 6/15
851165/851165 [==============================] - 294s 346u

In [11]:
# load json and create model
json_file = open("./models/{}".format(name + ".json"), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = ks.models.model_from_json(loaded_model_json)

model.load_weights("./models/{}".format(name + ".dat"))

In [13]:
# generate characters
for i in range(10):
    pattern_idx = np.random.randint(0, data_input_test.shape[0]-1)
    pattern = data_input_test[pattern_idx,:]
    print("")
    print("Input:")
    print(load_dataset.idx_vec_to_string(tokenizer.index_word, pattern))

    x = np.reshape(pattern, (1, pattern.shape[0]))

    prediction = np.squeeze(model.predict(x, verbose=0))
    for j in range(3):
        idx = np.argmax(prediction)
        print("Predicted class {} with probability {}: {}".format(j+1, prediction[idx], index_to_label_class[idx]))
        prediction[idx] = 0
        
    print("Actual class: {}".format(index_to_label_class[np.argmax(data_labels_test[pattern_idx])]))


Input:
an accident one day that would chill the heart of any man it would teach them not to drink a drop while the steering wheel in their hand this awful accident occurred on the 20th day of may and caused two little children to be sleepin beneath the clay these two little kids walked side by side along the state highway their poor old mother she had died and their daddy had ran away as these two little children walked arm in arm how sad their hearts did feel when around the curb came a speeding car with a drunk
Predicted class 1 with probability 0.4412298798561096: Country
Predicted class 2 with probability 0.1738954484462738: Rock
Predicted class 3 with probability 0.09098871052265167: Metal
Actual class: Country

Input:
of perilous buried to remain among the vanquished there lies a key for which you must retrieve but once within your soul belongs to fire illusive concepts attempts un obtained those who before you became the detained within the gates you seek fading light buried in

In [15]:
predictions = model.predict(data_input_test, verbose=0, batch_size=2048)

In [32]:
classifications = np.argmax(predictions, axis=1)
classifications_correct = np.argmax(data_labels_test, axis=1)
is_correct = classifications==classifications_correct
for i in range(0,7):
    acc = np.mean(is_correct[classifications_correct == i].astype(int))
    print("Per Class accuracy for {} is {}".format(index_to_label_class[i],acc))

Per Class accuracy for Country is 0.4891003251836685
Per Class accuracy for Pop is 0.5266063910552833
Per Class accuracy for Rock is 0.09959861850088678
Per Class accuracy for R&B is 0.082808181315644
Per Class accuracy for Hip-Hop is 0.8557089453389456
Per Class accuracy for Electronic is 0.09361022364217253
Per Class accuracy for Jazz is 0.0
